# Lesson 5 – Looping Pattern: Social Post Draft Refinement Agent

This notebook is part of the **LangGraph Agentic AI – Intro Course**.

In this final lesson of the core patterns, you will build a **looping agent** that iteratively improves a short social media post draft until it reaches a desired quality or hits a maximum number of iterations.


## 1. Objectives & Prerequisites

**Objectives**

By the end of this lesson, you can:

- Design a state that tracks **iterations** and **quality score**.
- Implement an **evaluator node** that scores a draft.
- Implement an **improver node** that modifies the draft to (hopefully) improve it.
- Use conditional edges in LangGraph to **loop** until a stop condition is met.

**Prerequisites**

- Lessons 1–4 completed (single-node, multiple inputs, sequential, conditional).
- Comfortable with basic loop logic (e.g., `while` loop idea) and simple scoring rules.


## 2. Environment Setup

Make sure you have installed the course dependencies (from the repo root):

```bash
python -m venv .venv
source .venv/bin/activate   # on Windows: .venv\Scripts\activate
pip install -r env/requirements.txt
```

Then start Jupyter and select the `langgraph-intro` (or equivalent) Python kernel.


## 3. Concept Warm-up

The **looping pattern** models an *iterative process* like:

1. Evaluate the current draft.
2. If it is not good enough and we haven't hit a limit:
   - Improve the draft a bit.
   - Go back to step 1.
3. Otherwise, stop and return the final draft.

Example initial post:

> "Learning AI. Any tips?"

We want the agent to:

- Make the post clearer or more engaging.
- Add a bit more context.
- Encourage comments.
- Stop once the quality is "good enough" or a maximum number of iterations is reached.


### Scratch cell (optional)

Use this cell for quick experiments while you follow along.


In [ ]:
# Scratch space

draft = "Learning AI. Any tips?"
len(draft)

## 4. Define the State

We'll use a `PostState` with fields to track the post and loop control:

- `draft: str` – current version of the post.
- `quality_score: int` – score in range 0–100.
- `iteration: int` – current iteration number.
- `max_iterations: int` – maximum allowed iterations.
- `history: List[str]` – previous drafts (for inspection/debugging).


In [ ]:
from typing import TypedDict, List

class PostState(TypedDict):
    draft: str
    quality_score: int
    iteration: int
    max_iterations: int
    history: List[str]

# Example initial state
initial_state: PostState = {
    "draft": "Learning AI. Any tips?",
    "quality_score": 0,
    "iteration": 0,
    "max_iterations": 3,
    "history": [],
}

initial_state

## 5. Evaluation Node – `evaluate_post_node`

This node assigns a simple quality score based on heuristics:

- Base score: 50.
- If the draft is very short (< 50 chars), subtract 20.
- If it includes a call to action (e.g., "comment", "let me know"), add 20.
- If it mentions learning or sharing, add 10.
- Clamp the score between 0 and 100.


In [ ]:
def evaluate_post_node(state: PostState) -> PostState:
    """Evaluate the current draft and update quality_score.

    This is a very simple, rule-based scoring function for teaching.
    """
    text = state["draft"]
    score = 50  # base score

    # Length penalty
    if len(text) < 50:
        score -= 20

    # Call-to-action bonus
    lower = text.lower()
    if "comment" in lower or "let me know" in lower:
        score += 20

    # Learning/sharing bonus
    if "learning" in lower or "sharing" in lower:
        score += 10

    # Clamp between 0 and 100
    score = max(0, min(100, score))
    state["quality_score"] = score
    return state


# Quick local test
test_state = initial_state.copy()
evaluate_post_node(test_state)

## 6. Improvement Node – `improve_post_node`

This node attempts to improve the draft:

- Append extra context if the draft is very short.
- Add a call to action if missing.
- Store the previous draft in `history`.
- Increment the `iteration` counter.


In [ ]:
def improve_post_node(state: PostState) -> PostState:
    """Improve the current draft based on simple rules.

    - Append more context if short
    - Add a call to action if none found
    - Track previous drafts in history
    - Increment iteration counter
    """
    draft = state["draft"]
    history = state["history"]

    # Save current draft to history
    history.append(draft)

    new_draft = draft
    lower = draft.lower()

    # If very short, extend with more detail
    if len(draft) < 50:
        new_draft += " I'm exploring this topic in depth and will share what I learn."

    # Add a call to action if missing
    if "comment" not in lower and "let me know" not in lower:
        new_draft += " Let me know your thoughts in the comments."

    state["draft"] = new_draft
    state["history"] = history
    state["iteration"] = state["iteration"] + 1
    return state


# Quick local test
test_state2 = initial_state.copy()
improve_post_node(test_state2)

## 7. Loop Logic – When to Continue vs Stop?

We want the graph to:

- **Continue looping** (evaluate → improve → evaluate → …) if:
  - `quality_score < 70`, and
  - `iteration < max_iterations`.

- **Stop** otherwise.

We'll express this as a **conditional edge** from the evaluation node:
- If condition == `"continue"` → go to `improve_post_node`.
- If condition == `"stop"` → go to `END`.

And then `improve_post_node` goes back to `evaluate_post_node`, forming a loop.


### 7.1 Build the LangGraph App

Let's wire the loop using `StateGraph` and `add_conditional_edges`.


In [ ]:
from langgraph.graph import StateGraph, END

# 1. Create the graph
graph = StateGraph(PostState)

# 2. Add nodes
graph.add_node("evaluate_post", evaluate_post_node)
graph.add_node("improve_post", improve_post_node)

# 3. Set entry point
graph.set_entry_point("evaluate_post")

# 4. Define the routing function

def should_continue(state: PostState) -> str:
    """Return "continue" or "stop" based on quality and iterations."""
    if state["quality_score"] < 70 and state["iteration"] < state["max_iterations"]:
        return "continue"
    return "stop"


# 5. Add conditional edges from evaluate_post
graph.add_conditional_edges(
    "evaluate_post",
    should_continue,
    {
        "continue": "improve_post",
        "stop": END,
    },
)

# 6. From improve_post, go back to evaluate_post
graph.add_edge("improve_post", "evaluate_post")

# 7. Compile the graph
app = graph.compile()

# 8. Test run
result = app.invoke(initial_state)
result

Try changing:

- `initial_state["draft"]`,
- `max_iterations`,

and rerun the invocation to see how many times the loop executes and what final draft is produced.

You can also inspect `result["history"]` to see the evolution of the draft.


## 8. Exercises

Use the cells below to implement the exercises for this lesson.

---

### Exercise 1 – Smarter Scoring

**Goal:**

Improve the scoring logic in `evaluate_post_node`:

- Add a bonus if the draft contains a question mark (`?`) to reward engagement.
- Add a bonus if it mentions a time horizon, e.g. words like `"today"`, `"this week"`, `"this year"`, `"over the next"`.

Keep the score clamped between 0 and 100.


In [ ]:
# TODO: Enhance evaluate_post_node with more scoring rules.

# Hint:
# if "?" in text:
#     score += something
# if any(phrase in lower for phrase in ["today", "this week", ...]):
#     score += something


### Exercise 2 – Stop Reason

**Goal:**

Add a new field to the state:

```python
stop_reason: str
```

Set it when the loop ends:

- If the loop stops because the quality is high enough, set:
  - `stop_reason = "good_quality"`.
- If it stops because `iteration >= max_iterations`, set:
  - `stop_reason = "max_iterations_reached"`.

You can implement this by updating the state inside `evaluate_post_node` before returning to `END`.


In [ ]:
# TODO: Add a stop_reason field and set it appropriately.

# Hint:
# - Extend PostState
# - In evaluate_post_node (or a helper), when you decide to stop, set stop_reason.


### Stretch Exercise 1 – User Feedback Loop

**Goal:**

Pretend a user can give feedback like `"shorter"` or `"more examples"`.

1. Add a `feedback: str` field to `PostState`.
2. Modify `improve_post_node` so that it behaves differently depending on `feedback`, for example:
   - If `feedback == "shorter"`, try to keep the draft from growing too long.
   - If `feedback == "more examples"`, append something like "For example, ...".

You can hard-code `feedback` in `initial_state` for this lesson.


In [ ]:
# TODO: Add a feedback field and update improve_post_node to react to it.

# Example feedback values: "shorter", "more examples", "neutral".


### Stretch Exercise 2 – Multi-platform Style

**Goal:**

Adapt the post style based on platform:

1. Add `platform: str` to the state, with values like `"linkedin"` or `"twitter"`.
2. Modify `improve_post_node` so that:
   - For `"twitter"`, it tries to keep the text shorter (e.g., avoid long additions).
   - For `"linkedin"`, it allows longer and slightly more formal text.

Again, you can hard-code `platform` in `initial_state` for now.


In [ ]:
# TODO: Add a platform field and adjust improvement behavior based on platform.

# Hint:
# if state["platform"] == "twitter":
#     # be more conservative in adding text


---

You’ve now completed the scaffold for **Lesson 5 – Looping Pattern**.

Next steps:

- Commit this notebook and your solutions to your Git repo.
- At this point, you’ve seen the core LangGraph patterns (single-node, multi-input, sequential, conditional, looping).
- You are ready to design a **capstone graph** that combines these ideas into a more realistic agent.
